In [167]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
from IPython.display import display, Markdown
import tiktoken
import os
import json
import random

load_dotenv(find_dotenv())

True

In [168]:
import os
from anthropic import Anthropic

client = Anthropic(
    # This is the default and can be omitted
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)


In [169]:
OpenAI_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
Anthropic_client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

In [170]:
def get_n_tokens(text):
    enc = tiktoken.encoding_for_model("gpt-4o")
    return len(enc.encode(text))

def get_openai_answer(instructions, prompt):
    completion = OpenAI_client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": instructions},
        {"role": "user", "content": prompt}
      ]
    )
    return completion.choices[0].message.content

def get_claude_answer(instructions, prompt):
    message = client.messages.create(
        max_tokens=2000,
        system = instructions,
        messages=[
            {"role": "user", "content": prompt}
        ],
        model="claude-3-5-sonnet-20240620",
        )
    return message.content[0].text

def read_file(file_path):
    try:
        with open(file_path, 'r') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return "File not found."

def read_all_files_in_folder(folder_path):
    contents_dict = {}
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                contents = file.read()
                contents_dict[filename] = contents
    return contents_dict

def get_cost_dollars(instructions, answer):
    input_1k_tokens = 0.005
    output_1k_tokens = 0.015
    return input_1k_tokens * get_n_tokens(instructions)/1000 + output_1k_tokens * get_n_tokens(answer)/1000

def render_markdown(text):
    display(Markdown(text))

def sample_dict(dictionary, sample_size):
    if sample_size > len(dictionary):
        raise ValueError("Sample size cannot be larger than the dictionary size")
    
    sampled_keys = random.sample(list(dictionary.keys()), sample_size)
    sampled_dict = {key: dictionary[key] for key in sampled_keys}
    
    return sampled_dict

In [171]:
ls

LLM_hook_generator_1.ipynb  hook_examples/
README.md                   hook_examples.json
Untitled.ipynb              instructions.txt


In [184]:
file_path = 'instructions.txt'
instructions = read_file(file_path)

folder_path = 'hook_examples'
file_to_code = read_all_files_in_folder(folder_path)

with open("hook_examples.json", 'r') as file:
    hook_examples_json = json.load(file)

In [185]:
all_contents.keys()

dict_keys(['LimitOrder.sol', 'WhiteListHook.sol', 'PointsHook.sol', 'GeomeanOracle.sol', 'MultiSigSwapHook.sol', 'KYCHook.sol', 'ERC721OwnershipHook.sol', 'CounterHook.sol', 'TakeProfitsHook.sol', 'GasPriceFeesHook.sol'])

In [196]:
final_instructions = instructions
counter = 1

hooks_for_instructions = sample_dict(hook_examples_json, 2)

for file, summary in (hooks_for_instructions).items():
    final_instructions += f"""----------\nHOOK EXAMPLE {counter}:\n\n
    SUMMARY: {summary}\n
    CODE:\n {file_to_code[file]}\n\n\n------------------\n\n\n"""
    counter+=1

final_instructions+="OUTPUT: ONLY Solidity code, nothing else - no explanations, summaries or descriptions. ONLY working Solidity code, WITH comments."

In [197]:
get_n_tokens(final_instructions)

2455

In [198]:
prompt = "I want a hook that sends an erc20 to the swapper (the hook will have a balance). If the balance is 0, do nothing. Also, have a whitelist"

#answer = gpt_get_answer(final_instructions, prompt)

In [199]:
answer = get_claude_answer(final_instructions, prompt)

In [201]:
render_markdown(answer)

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import {Hooks} from "@uniswap/v4-core/contracts/libraries/Hooks.sol";
import {BaseHook} from "v4-periphery/BaseHook.sol";
import {IPoolManager} from "@uniswap/v4-core/contracts/interfaces/IPoolManager.sol";
import {IERC20} from "@openzeppelin/contracts/token/ERC20/IERC20.sol";
import {SafeERC20} from "@openzeppelin/contracts/token/ERC20/utils/SafeERC20.sol";

contract RewardHook is BaseHook {
    using SafeERC20 for IERC20;

    // The ERC20 token to be sent as a reward
    IERC20 public immutable rewardToken;

    // Amount of tokens to be sent per swap
    uint256 public immutable rewardAmount;

    // Mapping to store whitelisted addresses
    mapping(address => bool) public whitelist;

    // Event emitted when a reward is sent
    event RewardSent(address indexed recipient, uint256 amount);

    constructor(
        IPoolManager _poolManager,
        IERC20 _rewardToken,
        uint256 _rewardAmount
    ) BaseHook(_poolManager) {
        rewardToken = _rewardToken;
        rewardAmount = _rewardAmount;
    }

    function getHooksCalls() public pure override returns (Hooks.Calls memory) {
        return Hooks.Calls({
            beforeInitialize: false,
            afterInitialize: false,
            beforeModifyPosition: false,
            afterModifyPosition: false,
            beforeSwap: false,
            afterSwap: true,
            beforeDonate: false,
            afterDonate: false
        });
    }

    // Function to add an address to the whitelist
    function addToWhitelist(address _address) external {
        whitelist[_address] = true;
    }

    // Function to remove an address from the whitelist
    function removeFromWhitelist(address _address) external {
        whitelist[_address] = false;
    }

    function afterSwap(
        address sender,
        IPoolManager.PoolKey calldata,
        IPoolManager.SwapParams calldata,
        int256 /* deltaAmount0 */,
        int256 /* deltaAmount1 */
    ) external override returns (bytes4) {
        // Check if the sender is whitelisted
        if (!whitelist[sender]) {
            return BaseHook.afterSwap.selector;
        }

        // Check the balance of the contract
        uint256 contractBalance = rewardToken.balanceOf(address(this));

        // If the balance is sufficient, send the reward
        if (contractBalance >= rewardAmount) {
            rewardToken.safeTransfer(sender, rewardAmount);
            emit RewardSent(sender, rewardAmount);
        }

        return BaseHook.afterSwap.selector;
    }

    // Function to withdraw any remaining tokens (in case the contract needs to be deprecated)
    function withdrawRemainingTokens(address recipient, uint256 amount) external {
        rewardToken.safeTransfer(recipient, amount);
    }
}

In [202]:
render_markdown(final_instructions)

You are an expert solidity developer.

Your task is to assist in creating hook smart contracts for Uniswap V4. 

Uniswap v4 Hooks -- also known simply as hooks -- are specially designed contracts that run at distinct points throughout a pool action's lifecycle. They serve as plugins allowing developers to tailor how pools, swaps, fees, and LP positions interact. This enables innovation atop Uniswap v4's core features, thereby supporting the development of custom AMM pools.

During the course of a pool action's lifecycle, a hook invokes custom logic primarily at four critical phases:

Initialize: Activated when the pool is deployed.
Modify Position: Used to add or remove liquidity.
Swap: Engages a swap between tokens within the V4 ecosystem.
Donate: Facilitates the donation of liquidity to a V4 pool. Upon initialization, a pool can be associated with a hook contract. Such a contract has the ability to execute any of the callback functions during the pool action's lifecycle:
{before,after}Initialize
{before,after}AddLiquidity
{before,after}RemoveLiquidity
{before,after}Swap
{before,after}Donate
Moreover, hooks can determine fees on swaps and liquidity withdrawals through callback functions. The flexibility lies in updating the fee values or callback logic based on the hook's design. Nevertheless, the callbacks activated on a pool, including the fee type, remain constant post pool initialization.

The available functions are:

beforeInitialize
afterInitialize
beforeAddLiquidity
beforeRemoveLiquidity
afterAddLiquidity
afterRemoveLiquidity
beforeSwap
afterSwap
beforeDonate
afterDonate

You are going to receive examples of hook contracts, with a SUMMARY and the CODE, separated by lines.

------------------

Here are some examples of hooks:


----------
HOOK EXAMPLE 1:


    SUMMARY: A unique hook making a Uniswap pool function as an oracle. The geomean price is calculated using the prices of the assets in the pool. This can be used to get a more accurate price for an asset than a single oracle.

    CODE:
 // SPDX-License-Identifier: UNLICENSED
pragma solidity ^0.8.19;

import {IPoolManager} from "@uniswap/v4-core/src/interfaces/IPoolManager.sol";
import {PoolId, PoolIdLibrary} from "@uniswap/v4-core/src/types/PoolId.sol";
import {Hooks} from "@uniswap/v4-core/src/libraries/Hooks.sol";
import {TickMath} from "@uniswap/v4-core/src/libraries/TickMath.sol";
import {Oracle} from "../../libraries/Oracle.sol";
import {BaseHook} from "../../BaseHook.sol";
import {PoolKey} from "@uniswap/v4-core/src/types/PoolKey.sol";
import {StateLibrary} from "@uniswap/v4-core/src/libraries/StateLibrary.sol";
import {BeforeSwapDelta, BeforeSwapDeltaLibrary} from "@uniswap/v4-core/src/types/BeforeSwapDelta.sol";

/// @notice A hook for a pool that allows a Uniswap pool to act as an oracle. Pools that use this hook must have full range
///     tick spacing and liquidity is always permanently locked in these pools. This is the suggested configuration
///     for protocols that wish to use a V3 style geomean oracle.
contract GeomeanOracle is BaseHook {
    using Oracle for Oracle.Observation[65535];
    using PoolIdLibrary for PoolKey;
    using StateLibrary for IPoolManager;

    /// @notice Oracle pools do not have fees because they exist to serve as an oracle for a pair of tokens
    error OnlyOneOraclePoolAllowed();

    /// @notice Oracle positions must be full range
    error OraclePositionsMustBeFullRange();

    /// @notice Oracle pools must have liquidity locked so that they cannot become more susceptible to price manipulation
    error OraclePoolMustLockLiquidity();

    /// @member index The index of the last written observation for the pool
    /// @member cardinality The cardinality of the observations array for the pool
    /// @member cardinalityNext The cardinality target of the observations array for the pool, which will replace cardinality when enough observations are written
    struct ObservationState {
        uint16 index;
        uint16 cardinality;
        uint16 cardinalityNext;
    }

    /// @notice The list of observations for a given pool ID
    mapping(PoolId => Oracle.Observation[65535]) public observations;
    /// @notice The current observation array state for the given pool ID
    mapping(PoolId => ObservationState) public states;

    /// @notice Returns the observation for the given pool key and observation index
    function getObservation(PoolKey calldata key, uint256 index)
        external
        view
        returns (Oracle.Observation memory observation)
    {
        observation = observations[PoolId.wrap(keccak256(abi.encode(key)))][index];
    }

    /// @notice Returns the state for the given pool key
    function getState(PoolKey calldata key) external view returns (ObservationState memory state) {
        state = states[PoolId.wrap(keccak256(abi.encode(key)))];
    }

    /// @dev For mocking
    function _blockTimestamp() internal view virtual returns (uint32) {
        return uint32(block.timestamp);
    }

    constructor(IPoolManager _manager) BaseHook(_manager) {}

    function getHookPermissions() public pure override returns (Hooks.Permissions memory) {
        return Hooks.Permissions({
            beforeInitialize: true,
            afterInitialize: true,
            beforeAddLiquidity: true,
            beforeRemoveLiquidity: true,
            afterAddLiquidity: false,
            afterRemoveLiquidity: false,
            beforeSwap: true,
            afterSwap: false,
            beforeDonate: false,
            afterDonate: false,
            beforeSwapReturnDelta: false,
            afterSwapReturnDelta: false,
            afterAddLiquidityReturnDelta: false,
            afterRemoveLiquidityReturnDelta: false
        });
    }

    function beforeInitialize(address, PoolKey calldata key, uint160, bytes calldata)
        external
        view
        override
        onlyByManager
        returns (bytes4)
    {
        // This is to limit the fragmentation of pools using this oracle hook. In other words,
        // there may only be one pool per pair of tokens that use this hook. The tick spacing is set to the maximum
        // because we only allow max range liquidity in this pool.
        if (key.fee != 0 || key.tickSpacing != manager.MAX_TICK_SPACING()) revert OnlyOneOraclePoolAllowed();
        return GeomeanOracle.beforeInitialize.selector;
    }

    function afterInitialize(address, PoolKey calldata key, uint160, int24, bytes calldata)
        external
        override
        onlyByManager
        returns (bytes4)
    {
        PoolId id = key.toId();
        (states[id].cardinality, states[id].cardinalityNext) = observations[id].initialize(_blockTimestamp());
        return GeomeanOracle.afterInitialize.selector;
    }

    /// @dev Called before any action that potentially modifies pool price or liquidity, such as swap or modify position
    function _updatePool(PoolKey calldata key) private {
        PoolId id = key.toId();
        (, int24 tick,,) = manager.getSlot0(id);

        uint128 liquidity = manager.getLiquidity(id);

        (states[id].index, states[id].cardinality) = observations[id].write(
            states[id].index, _blockTimestamp(), tick, liquidity, states[id].cardinality, states[id].cardinalityNext
        );
    }

    function beforeAddLiquidity(
        address,
        PoolKey calldata key,
        IPoolManager.ModifyLiquidityParams calldata params,
        bytes calldata
    ) external override onlyByManager returns (bytes4) {
        int24 maxTickSpacing = manager.MAX_TICK_SPACING();
        if (
            params.tickLower != TickMath.minUsableTick(maxTickSpacing)
                || params.tickUpper != TickMath.maxUsableTick(maxTickSpacing)
        ) revert OraclePositionsMustBeFullRange();
        _updatePool(key);
        return GeomeanOracle.beforeAddLiquidity.selector;
    }

    function beforeRemoveLiquidity(
        address,
        PoolKey calldata,
        IPoolManager.ModifyLiquidityParams calldata,
        bytes calldata
    ) external view override onlyByManager returns (bytes4) {
        revert OraclePoolMustLockLiquidity();
    }

    function beforeSwap(address, PoolKey calldata key, IPoolManager.SwapParams calldata, bytes calldata)
        external
        override
        onlyByManager
        returns (bytes4, BeforeSwapDelta, uint24)
    {
        _updatePool(key);
        return (GeomeanOracle.beforeSwap.selector, BeforeSwapDeltaLibrary.ZERO_DELTA, 0);
    }

    /// @notice Observe the given pool for the timestamps
    function observe(PoolKey calldata key, uint32[] calldata secondsAgos)
        external
        view
        returns (int56[] memory tickCumulatives, uint160[] memory secondsPerLiquidityCumulativeX128s)
    {
        PoolId id = key.toId();

        ObservationState memory state = states[id];

        (, int24 tick,,) = manager.getSlot0(id);

        uint128 liquidity = manager.getLiquidity(id);

        return observations[id].observe(_blockTimestamp(), secondsAgos, tick, state.index, liquidity, state.cardinality);
    }

    /// @notice Increase the cardinality target for the given pool
    function increaseCardinalityNext(PoolKey calldata key, uint16 cardinalityNext)
        external
        returns (uint16 cardinalityNextOld, uint16 cardinalityNextNew)
    {
        PoolId id = PoolId.wrap(keccak256(abi.encode(key)));

        ObservationState storage state = states[id];

        cardinalityNextOld = state.cardinalityNext;
        cardinalityNextNew = observations[id].grow(cardinalityNextOld, cardinalityNext);
        state.cardinalityNext = cardinalityNextNew;
    }
}


------------------


----------
HOOK EXAMPLE 2:


    SUMMARY: Disallows a swap if sender doesn't own an ERC721 NFT. This can be used to prevent people from selling tokens that they don't own.

    CODE:
 // SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import {Hooks} from "@uniswap/v4-core/contracts/libraries/Hooks.sol";
import {BaseHook} from "v4-periphery/BaseHook.sol";
import {IPoolManager} from "@uniswap/v4-core/contracts/interfaces/IPoolManager.sol";
import {IERC721} from "@openzeppelin/contracts/token/ERC721/IERC721.sol";

/// @title ERC721OwnershipHook
contract ERC721OwnershipHook is BaseHook {
    /// @notice NFT contract
    IERC721 public immutable nftContract;

    error NotNftOwner();

    constructor(
        IPoolManager _poolManager,
        IERC721 _nftContract
    ) BaseHook(_poolManager) {
        nftContract = _nftContract;
    }

    function getHooksCalls() public pure override returns (Hooks.Calls memory) {
        return
            Hooks.Calls({
                beforeInitialize: false,
                afterInitialize: false,
                beforeModifyPosition: false,
                afterModifyPosition: false,
                beforeSwap: true,
                afterSwap: false,
                beforeDonate: false,
                afterDonate: false
            });
    }

    function beforeSwap(
        address sender,
        IPoolManager.PoolKey calldata,
        IPoolManager.SwapParams calldata
    ) external override returns (bytes4) {
        if (nftContract.balanceOf(sender) == 0) {
            revert NotNftOwner();
        }

        return BaseHook.beforeSwap.selector;
    }
}


------------------


OUTPUT: ONLY Solidity code, nothing else - no explanations, summaries or descriptions. ONLY working Solidity code, WITH comments.